In [1]:
import importlib
import pandas as pd
import numpy as np
import re

In [2]:
from functions.get_mon_data import *
from functions.functions_dataframe import * 
from functions.functions_shared import *
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

In [ ]:
""" 
Для формування списку ЄДРПОУ навчальних закладів, які потрібно наносити на мапу, використані бази даних МОН, опубліковані у листопаді 2019 року на Порталі відкритих даних.

### Перелік закладів загальної середньої та дошкільної освіти (ЗЗСДО)
    https://data.gov.ua/dataset/bd70757a-ab49-46be-a650-608a9b3bd7b1
    
### Перелік закладів позашкільної освіти (державна форма власності)
    https://data.gov.ua/dataset/9a88fb96-a043-4769-acf1-1c4a948ec285
    Перелік закладів позашкільної освіти системи Міністерства освіти і науки України
    https://data.gov.ua/dataset/18d7fb70-3781-4017-88d2-651e14d14211
    Заклади професійної (професійно-технічної) освіти
    https://data.gov.ua/dataset/824513a9-aa5e-45c8-b45e-dced6909761a
"""

In [ ]:
"""
df_raw - база даних завершених закупівель 24 обласних центрів з 1 серпня 2016 до 31 грудня 2019.
В Донецькій області - Краматорськ. В Луганській - Сєвєродонецьк. 
Згерерована у Профейсійному модулі аналітики bipro.prozorro.
"ИдентификаторЛота" є унікальним ідентифікатором рядка.
"""  

In [ ]:
df_raw = pandas.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep = ';', low_memory = False)
training_data_columns = df_raw.columns.tolist()
print("Дані з файлу викачки BIPro Prozorro: " + "df_raw" + str(df_raw.shape))

##### Step 0: Підготовка бази МОН для використання у фільтруванні закупівель

In [ ]:
expdata_sc = pandas.read_csv('data/mon_data/expdata_sc.csv', sep = ';', low_memory = False)
expdata_dnz = pandas.read_csv('data/mon_data/expdata_dnz.csv', sep = ';', low_memory = False)

In [ ]:
expdata_sc = expdata_sc.fillna(0)
expdata_sc['edrpou'] = expdata_sc['edrpou'].astype(np.int64)
mask = (expdata_sc['edrpou'].astype(str).str.len() == 7) | (expdata_sc['edrpou'].astype(str).str.len() == 8)
expdata_sc = expdata_sc.loc[mask].sort_values(by=['edrpou'])
print("expdata_sc", expdata_sc.shape)

In [ ]:
expdata_dnz['edrpou'] = pd.to_numeric(expdata_dnz['edrpou'],errors='coerce')
expdata_dnz = expdata_dnz.fillna(0)
expdata_dnz['edrpou'] = expdata_dnz['edrpou'].astype(np.int64)
mask = (expdata_dnz['edrpou'].astype(str).str.len() == 7) | (expdata_dnz['edrpou'].astype(str).str.len() == 8)
expdata_dnz = expdata_dnz.loc[mask].sort_values(by=['edrpou'])
print("expdata_dnz", expdata_dnz.shape)

In [ ]:
frames = [expdata_dnz, expdata_sc]
df_mon_data = pd.concat(frames)
print("База МОН шкіл і садків з 7-8 значинми ЄДРПОУ, df_mon_data", df_mon_data.shape)

In [ ]:
filter_keywords_exclude = ["департамент", "управлін.*освіт", "відділ.*освіт"]
df_raw_upravlin_osvity = df_raw[df_raw['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_exclude)), case = False, na = False, regex = True)]
edrpou_upravlin_osvity_prozorro = df_raw_upravlin_osvity['IDOrganizator'].unique().tolist()
print("edrpou_upravlin_osvity_prozorro", len(edrpou_upravlin_osvity_prozorro))

mask = (df_mon_data['edrpou'].isin(edrpou_upravlin_osvity_prozorro))
df_mon_data_bez_upravlin_osvity = df_mon_data.loc[~mask]
print("df_mon_data_bez_upravlin_osvity", df_mon_data_bez_upravlin_osvity.shape)

##### Step 1: Створити базу навчальних закладів, які самостійно купують через ProZorro. Виділення закупівель шкіл-замовників , згідно з базою МОН (пошук по колонці "IDOrganizator"

In [ ]:
mon_data = df_mon_data_bez_upravlin_osvity.edrpou.unique().tolist()

df_tenders_by_schools_by_edrpou = df_raw[df_raw['IDOrganizator'].isin(mon_data)]
df_undefined_1 = df_raw[~df_raw['IDOrganizator'].isin(mon_data)]

print("Закупівлі шкіл-замовників, визначених через МОН: df_tenders_by_schools_by_edrpou " + str(df_tenders_by_schools_by_edrpou.shape))
print("Невизначені записи: df_undefined_1 " + str(df_undefined_1.shape))

In [ ]:
# evaluation of results of Step 1

# df_tenders_by_schools_by_edrpou['Организатор'].value_counts()

##### Step 2: Виділення закупівель шкіл-замовників за допомогою пошуку ключових слів в колонці "Организатор" 

In [ ]:
df_tenders_by_schools_by_keywords = df_undefined_1[df_undefined_1['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), case = False, na = False, regex = True) ]
print(f"Закупівлі шкіл-замовників, визначених через 'Организатор': df_tenders_by_schools_by_keywords {df_tenders_by_schools_by_keywords.shape}")

In [ ]:
# evalutaion #1 of results of Step 2
# df_tenders_by_schools_by_keywords['Организатор'].value_counts()

In [ ]:
# Додаткове фільтрування df_tenders_by_schools_by_keywords

filter_keywords_step_2_exclude = ['професійн', 'міжшкільн']
df_tenders_by_schools_by_keywords_2_temp = df_tenders_by_schools_by_keywords[~df_tenders_by_schools_by_keywords['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_step_2_exclude)), case = False, na = False, regex = True)]

filter_keywords_step_2_include = ["школа І-І.*ступенів", "навчальн.*виховн.*комплекс", "гімназ", "ліце", "дошкільний"]
df_tenders_by_schools_by_keywords_2 = df_tenders_by_schools_by_keywords_2_temp[df_tenders_by_schools_by_keywords_2_temp['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_step_2_include)), case = False, na = False, regex = True)]
print(f"Видалення із датафрейму df_tenders_by_schools_by_keywords зайвих закупівель, {df_tenders_by_schools_by_keywords_2_temp.shape} \nЗалишилось: {df_tenders_by_schools_by_keywords_2.shape}")

In [ ]:
# evalutaion #2 of results of Step 2
# df_tenders_by_schools_by_keywords_2['Организатор'].value_counts()

##### Step 2.5: Об'єднання двох датафреймів шкіл-замовників в один

In [ ]:
df_tenders_by_schools = pandas.concat([df_tenders_by_schools_by_edrpou, 
                                       df_tenders_by_schools_by_keywords_2])
print(f"Закупівлі шкіл-замовників: df_tenders_by_schools {df_tenders_by_schools.shape} \nНевизначені записи: df_undefined_1:{df_undefined_1.shape}")

##### Step 3: Класифікація шкіл-замовників по типах

In [ ]:
columns_to_classify = ['Организатор']
df_tenders_by_schools_classified = classification_by_type_one_column(df_tenders_by_schools, 
                                                                     columns_to_classify, 
                                                                     filter_priority, 
                                                                     filter_name_priority)

In [ ]:
# evaluation of Step 3

# pd.set_option("display.max_colwidth", 200)
# print("Заклади, де не визначений тип:")
# df_tenders_by_schools_classified[df_tenders_by_schools_classified['Тип закладу'] == 'Невідомо']['Организатор'].unique().tolist()

In [ ]:
# EXPERIMETNS FOR SOLUTION #3 FROM STEP 11 

def classification_by_type_one_column_TEST(df, columns, priority, names):
    given_df = df[:]
    checked_priority = priority[::-1]
    checked_names = names[::-1]
    given_df['Тип закладу_2'] = "Невідомо"

    for column in columns:
        given_df[column + "_check"] = ""

    for idx in range(len(checked_priority)):
        checked_filter = checked_priority[idx]
        for column in columns:
            given_df[column + "_check"] = given_df[column].astype(str).str.contains(r'({})'.format('|'.join(checked_filter)),
                case = False, na = False, regex = True)

        checked_name = checked_names[idx]
        given_df['Тип закладу_2'] = np.where(given_df['Организатор_check'] == True, checked_name, given_df['Тип закладу_2'])
    return given_df


filter_priority = [
        filter_specialna,
        filter_nvk,
        filter_gimnasium,
        filter_liceum,
        filter_specializovana,
        filter_internat,
        filter_kombinat,
        filter_sanatorna,
        filter_specializovana,
        filter_sadik,
        filter_shkola 
    ]

filter_name_priority = [
        filter_specialna_name,
        filter_nvk_name,
        filter_gimnasium_name,
        filter_liceum_name,
        filter_specializovana_name,
        filter_internat_name,
        filter_kombinat_name,
        filter_sanatorna_name,
        filter_specializovana_name,
        filter_sadik_name,
        filter_shkola_name
    ]

filter_vechirni = ['вечір.*змінн', 'вечір.*школ']
filter_specialna = ['cпеціальн']
filter_sanatorna = ['санаторн.*шк', '.*шк.*санаторн']
filter_kombinat = ['міжшкільн.*комбінат']
filter_internat = ['інтернат']
filter_nvk = ['центр.*розвитк.*дитин', 'увк', 'нвк', 'навчальн.*виховн.*комплекс', 'нво', 'навчальн.*виховн.*об']
filter_gimnasium = ['гімназі[яї]']
filter_liceum = ['ліце[йяю]']
filter_specializovana = ['спеціалізован']
filter_shkola = ['загальноосвітн', 'І.*ступен', 'колегіум', 'зош', 'сзш', 'загальн.*середн', 'початков.*школ']
filter_sadik = ['днз', 'садок', 'садка' 'дошкільн', 'ясла', 'ясел']

In [ ]:
columns_to_classify = ['Организатор']

df_error_3_test = df_error_3[df_error_3['Заклад-замовник'] == True].copy()

df_error_3_classified = classification_by_type_one_column_TEST(df_error_3_test, 
                                                               columns_to_classify, 
                                                               filter_priority_2, 
                                                               filter_name_priority_2)

df_error_3_classified[['Организатор', 'Тип закладу', 'Тип закладу_2']]

##### Step 4: Нумерація шкіл-замовників

In [ ]:
columns_to_numerate = ['Организатор']
df_tenders_by_schools_numerated = numeration_one_column(df_tenders_by_schools_classified, 
                                                        columns_to_numerate, 
                                                        reg_number)

##### Step 4.5: Найменування шкіл-замовників

In [ ]:
columns_to_name = ['Организатор']
df_tenders_by_schools_named = naming_one_column(df_tenders_by_schools_numerated, 
                                                columns_to_name, 
                                                reg_quotes)

In [ ]:
print('Було закупівель', df_tenders_by_schools_named.shape)
df_tenders_by_schools_named = df_tenders_by_schools_named[~df_tenders_by_schools_named['Организатор'].astype(str).str.contains(r'({})'.format('|'.join(filter_poza)),
                case = False, na = False, regex = True)]
df_tenders_by_schools_named['Номер'] = df_tenders_by_schools_named['Номер'].replace(r'^\s*$', np.nan, regex=True) # перетворити пусті строки в NaN
print(f'Залишилось закупівель після видалення закупівель закладів ключовими словами {filter_poza}, \ndf_tenders_by_schools_named', df_tenders_by_schools_named.shape)

In [ ]:
# # evaluation of Step 4

# for_analysis_step_4 = df_tenders_by_schools_named[df_tenders_by_schools_named['Назва'].isnull() & 
#                                   df_tenders_by_schools_named['Номер'].isnull()]
# print('Навчальні заклади, де не визначились номер чи назва:')
# for_analysis_step_4['Организатор'].unique().tolist()

##### Step 4.9: Форматування бази шкіл-замовників

In [ ]:
df_tenders_by_schools = df_tenders_by_schools_named.copy()
df_tenders_by_schools['Заклад-замовник'] = True

col_names = training_data_columns.copy() 
col_names += ['Номер', 'Назва', 'Тип закладу', 'Заклад-замовник']

df_tenders_by_schools = df_tenders_by_schools[col_names]

##### Step 5: Виділення закупівель шкіл-не-замовників за допомогою пошуку ключових слів в колонках Тендер, ОписаниеТендера, Лот

In [ ]:
df_tenders_not_by_schools_by_keywords = df_undefined_1[df_undefined_1['Тендер'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_extended)), 
            case = False, na = False, regex = True)  | 
                                                      df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_extended)), 
            case = False, na = False, regex = True) | 
                                                      df_undefined_1['Лот'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords_extended)), 
            case = False, na = False, regex = True) | df_undefined_1['Тендер'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict_extended)), 
            case = False, na = False, regex = True) | df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict_extended)), 
            case = False, na = False, regex = True) | df_undefined_1['Лот'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict_extended)), 
            case = False, na = False, regex = True)]

print("Закупівлі шкіл-не-замовників, визначених через Тендер, Лот, ОписаниеТендера: df_tenders_by_schools_by_keywords " + str(df_tenders_not_by_schools_by_keywords.shape))

In [ ]:
# evaluation of Step 5

# df_tenders_not_by_schools_by_keywords

##### Step 6: Класифікація шкіл-не-замовників по типах

In [ ]:
columns_to_classify = ['Тендер', 'ОписаниеТендера', 'Лот']
df_tenders_not_by_schools_classified = classification_by_type_three_cols(df_tenders_not_by_schools_by_keywords,
                                                                         columns_to_classify, 
                                                                         filter_priority, 
                                                                         filter_name_priority)

In [ ]:
# # evaluation of Step 6

# pd.set_option("display.max_colwidth", 200)

# type_sample = 'Інтернат'
# df_tenders_not_by_schools_classified[df_tenders_not_by_schools_classified['Тип закладу'] == type_sample][['Тендер', 'ОписаниеТендера', 'Лот', 'Тип закладу']].sort_values(by=['Тип закладу'])
# df_tenders_not_by_schools_classified['Тип закладу'].value_counts()
# df_tenders_not_by_schools_classified.sample(5)

##### Step 7: Нумерація шкіл-не-замовників

In [ ]:
"""
Принцип фільтрування в функції numeration():

    Визначається наявністья символу № в кожній колонці columns_to_numerate_step7
    
    Регулярний вираз знаходить всі паттерни і джойнить через кому
    
    Паралельно створюється колонка, де рахується кількість знайдених паттернів
    
    Результат фільтрування numeration() - потрапляє у датафрейм df_step7_numerated
"""
columns_to_numerate_step7 = ['Тендер', 'Лот', 'ОписаниеТендера']
action_mark = 'numeration'

df_step7 = df_tenders_not_by_schools_classified[df_tenders_not_by_schools_classified['Тип закладу'] != 'Невідомо'].copy()
df_step7['ОписаниеТендера'] = df_step7['ОписаниеТендера'].fillna('')
df_step7_numerated = numeration(df_step7, columns_to_numerate_step7, reg_number, action_mark)
print("df_step7_numerated", df_step7_numerated.shape)

In [ ]:
"""
    Якщо в колонках Лот і Тендер знайшло по одному паттерну і вони однакові - 
    закупівля вважається такою, де коректно визначений номер

    Ці закупівлі потрапляють до датафрейму df_step7_numerated_lot_tender

    Колонка "Номер" отримує знайдений номер "Лот_numeration"

    Решта закупівель потрапляють до датафрейму df_step7_numerated_undefined.
"""

df_step7_numerated['Номер'] = np.where(((df_step7_numerated['Тендер_numeration_count'] == 1) | 
                                        (df_step7_numerated['Лот_numeration_count'] == 1)) & 
                                        (df_step7_numerated['Лот_numeration_count'] == df_step7_numerated['Тендер_numeration_count']), 
                                        df_step7_numerated['Лот_numeration'], 
                                        'Невідомо')

df_step7_numerated_lot_tender = df_step7_numerated[df_step7_numerated['Номер'] != 'Невідомо']

df_step7_numerated_undefined = df_step7_numerated[df_step7_numerated['Номер'] == 'Невідомо']

print("df_step7_numerated_lot_tender", df_step7_numerated_lot_tender.shape, "\ndf_step7_numerated_undefined", df_step7_numerated_undefined.shape)

In [ ]:
"""
    Якщо в колонці Лот знайдений один паттерн - 
    закупівля потрапляють до датафрейму df_step7_numerated_lot
    
    Решта закупівель потрапляють до датафрейму до df_step7_numerated_undefined_2
"""

df_step7_numerated_lot = df_step7_numerated_undefined.copy()

df_step7_numerated_lot['Номер'] = np.where(df_step7_numerated_lot['Лот_numeration_count'] == 1, 
                                                 df_step7_numerated_lot['Лот_numeration'], 
                                                 'Невідомо')

df_step7_numerated_undefined_2 = df_step7_numerated_lot[df_step7_numerated_lot['Номер'] == 'Невідомо']
df_step7_numerated_lot = df_step7_numerated_lot[df_step7_numerated_lot['Номер'] != 'Невідомо']

print("df_step7_numerated_lot", df_step7_numerated_lot.shape, "\ndf_step7_numerated_undefined_2", df_step7_numerated_undefined_2.shape)

In [ ]:
"""
    Якщо в колонці Описаниетендера знайдений один паттерн - 
    закупівля потрапляють до датафрейму df_step7_numerated_opystendera
    
    Решта закупівель потрапляють до датафрейму до df_step7_numerated_undefined_3
"""

df_step7_numerated_opystendera = df_step7_numerated_undefined_2.copy()

df_step7_numerated_opystendera['Номер'] = np.where(df_step7_numerated_opystendera['ОписаниеТендера_numeration_count'] == 1, 
                                                 df_step7_numerated_opystendera['ОписаниеТендера_numeration'], 
                                                 'Невідомо')

df_step7_numerated_undefined_3 = df_step7_numerated_opystendera[df_step7_numerated_opystendera['Номер'] == 'Невідомо']
df_step7_numerated_opystendera = df_step7_numerated_opystendera[df_step7_numerated_opystendera['Номер'] != 'Невідомо']

print("df_step7_numerated_opystendera", df_step7_numerated_opystendera.shape, "\df_step7_numerated_undefined_3", df_step7_numerated_undefined_3.shape)

In [ ]:
"""
    Якщо в колонці Тендер знайдений один паттерн - 
    закупівля потрапляють до датафрейму df_step7_numerated_tender
    
    Решта закупівель потрапляють до датафрейму до df_step7_numerated_undefined_4
"""
df_step7_numerated_tender = df_step7_numerated_undefined_3.copy()

df_step7_numerated_tender['Номер'] = np.where(df_step7_numerated_tender['Тендер_numeration_count'] == 1, 
                                                 df_step7_numerated_tender['Тендер_numeration'], 
                                                 'Невідомо')
df_step7_numerated_undefined_4 = df_step7_numerated_tender[df_step7_numerated_tender['Номер'] == 'Невідомо']
df_step7_numerated_tender = df_step7_numerated_tender[df_step7_numerated_tender['Номер'] != 'Невідомо']

print("df_step7_numerated_tender", df_step7_numerated_tender.shape, "\ndf_step7_numerated_undefined_3", df_step7_numerated_undefined_3.shape)

##### Step 7.5: Об'єднання датафреймів шкіл-не-замовників з номерами

In [ ]:
frames = [
            df_step7_numerated_lot_tender, 
            df_step7_numerated_lot, 
            df_step7_numerated_opystendera, 
            df_step7_numerated_tender
         ]

df_tenders_not_by_schools_numerated = pd.concat(frames)
df_tenders_not_by_schools_numerated['Назва'] = np.nan # Створення пустої колонки "Назва" для подальшого обєднання з датафреймом з номерами

##### Step 8: Найменування шкіл-не-замовників

In [ ]:
columns_to_classify_step8 = ['Тендер', 'Лот']
action_mark = 'naming'

df_tenders_not_by_schools_named = naming(df_step8, columns_to_classify_step8, reg_quotes, action_mark) 
df_tenders_not_by_schools_named['Назва'] = df_tenders_not_by_schools_named['Тендер_naming'].apply(lambda loc: loc if loc else loc['Лот_naming'])
df_tenders_not_by_schools_named['Назва'] = df_tenders_not_by_schools_named['Назва'].fillna('Школа невідома')

print("df_tenders_not_by_schools_named", df_tenders_not_by_schools_named.shape)

In [ ]:
# EXPERIMETNS FOR SOLUTION #26 FROM STEP 11 

# # Як поєднати два способи?

# findall збирає всі співпадіння і залишає перше [0]
def naming_test_1(given_df, columns_to_name, regex, action_mark):
    for column in columns_to_name:
        given_df[column + '_' + action_mark] = given_df[column].str.findall(f"{regex}", 
                                                                            flags = re.
                                                                            IGNORECASE).str[0].str[0]
        given_df[column + '_' + action_mark] = given_df[column + '_' + action_mark].replace(r'^\s*$', np.nan, regex=True)
    return given_df

# findall збирає всі співпадіння і залишає останнє [-1]
def naming_test_2(given_df, columns_to_name, regex, action_mark):
    for column in columns_to_name:
        given_df[column + '_' + action_mark] = given_df[column].str.findall(f"{regex}", 
                                                                            flags = re.
                                                                            IGNORECASE).str[0].str[-1]
        given_df[column + '_' + action_mark] = given_df[column + '_' + action_mark].replace(r'^\s*$', np.nan, regex=True)
    return given_df

In [ ]:
# чому порядок OR має значення?
reg_quotes = '([«](.*?)[»]{1}?)|(["](.*?)["]{1}?)'
reg_quotes_2 = '(["](.*?)["]{1}?)|([«](.*?)[»]{1}?)'

df_step8 = df_step7_numerated_undefined_4.copy()

df_tenders_not_by_schools_named = naming_test_1(df_step8, 
                                                columns_to_classify_step8, 
                                                reg_quotes, 
                                                action_mark) 
df_tenders_not_by_schools_named[df_tenders_not_by_schools_named['Лот_naming'].notnull()].shape

In [ ]:
# df_tenders_not_by_schools_named[df_tenders_not_by_schools_named['Лот_naming'].notnull()]['Лот_naming']
# df_tenders_not_by_schools_named['Лот_naming'].unique().tolist()

##### Step 9: Об'єднання двох датафреймів шкіл-не-замовників

In [ ]:
frames = [df_tenders_not_by_schools_numerated, df_tenders_not_by_schools_named]
df_tenders_not_by_schools = pd.concat(frames)

In [ ]:
df_tenders_not_by_schools['Заклад-замовник'] = False

In [ ]:
df_tenders_not_by_schools = df_tenders_not_by_schools[col_names]
print("df_tenders_not_by_schools_numerated", df_tenders_not_by_schools_numerated.shape)

##### Step 9.5: Створення єдиної колонки для Номеру і Назви - Номер/Назва (ЩЕ ПІД ПИТАННЯМ)

In [ ]:
# """
# У датафреймі Шкіл-замовників df_tenders_by_schools_temp в рядках, 
#     - де Номер не визначився, присвоюється Назва
#     - де Назва не визначилась, присвоюється поле Організатор
# """

# df_tenders_by_schools_temp = df_tenders_by_schools.copy()
# df_tenders_by_schools_temp['Номер/Назва'] = np.where(df_tenders_by_schools_temp['Номер'].notnull(), df_tenders_by_schools_temp['Номер'], df_tenders_by_schools_temp['Назва'])
# df_tenders_by_schools_temp['Номер/Назва'] = np.where(df_tenders_by_schools_temp['Номер/Назва'].isnull(), df_tenders_by_schools_temp['Организатор'], df_tenders_by_schools_temp['Номер/Назва'])

In [ ]:
# """
# У датафреймі Шкіл-не-замовників df_tenders_not_by_schools в рядках, 
#     - де Номер не визначився, присвоюється Назва
#     - де Назва не визначилась, присвоюється поле Організатор

# В колонці 'Номер/Назва' значення "Невідомо" фактично означає закупівлю для кількох закладів освіти

# """

# df_tenders_not_by_schools_temp = df_tenders_not_by_schools.copy()
# df_tenders_not_by_schools_temp['Номер/Назва'] = np.where(df_tenders_not_by_schools_temp['Номер'].notnull(), df_tenders_not_by_schools_temp['Номер'], df_tenders_not_by_schools_temp['Назва'])

##### Step 10: Об'єднання датафреймів шкіл-не-замовників та шкіл-замовників

In [3]:
frames = [df_tenders_by_schools, df_tenders_not_by_schools]
df_tenders = pd.concat(frames)
print("df_tenders", df_tenders.shape)

NameError: name 'df_tenders_by_schools' is not defined

In [ ]:
df_tenders_filtered_1 = df_tenders.sample(1000)
df_tenders_filtered_1['Correct'] = 'undefined'
print("df_tenders_filtered_1:", df_tenders_filtered_1.shape, "\nCols:", df_tenders_filtered_1.columns)
# df_tenders_filtered_1.to_csv('results/school-procurements-1000-random.csv', sep=';')

#### Step 11: RandomDataResuls

In [4]:
"""
    Колонка Correct:
    
        1 - коректо визначено
    
    Некоректно:
        
        2 - номер визначений неправильно:
            
            21 - відсутній символ №
            22 - кілька символів №
            23 - без лапок
            24 - номер чи назва в колонці Описание Тендер
            25 - лапки не визначились
            26 - кілька лапок
        
        3 - неправильно тип закладу
"""

random_1000_results = pandas.read_csv('results/school-procurements-1000-random-marked.csv', index_col = 0, error_bad_lines=False, sep = ',', low_memory = False)
random_1000_results = random_1000_results.iloc[:, :-1]
print(random_1000_results.shape)

(1000, 8)


In [ ]:
# random_1000_results.index.values.tolist()
random_1000_results.Correct.value_counts()

In [ ]:
"""
TODO
"""

In [ ]:
""" UNDONE

    Correct mark #21:
        Зміни у Step 7:
            мабуть рішенням є регулярний вираз, який би відбирав 1-3 значне число,
            яке знаходить відразу після ключового слова.
            Можливо відстань рахувати між знайденими ключовими словами?
                приклад index 4337: "Капітальний ремонт будівлі ДНЗ 126"
                
    
    replace 'ДНЗ ' - 'ДНЗ №'
            
"""
# random_1000_results[random_1000_results.Correct=='21']

In [ ]:
""" UNDONE

    Correct mark #22:
        Зміни у Step 7:
            мабуть рішенням є регулярний вираз, який би шукав число з символом №,
            яке знаходиться ПІСЛЯ знайденого ключового слова, а не ДО. І він має бути тільки один.
            Можливо відстань рахувати між знайденими ключовими словами?
                приклад index 78075: "Лот №58 крупи та борошно, НВК №8"
            
"""
# random_1000_results[random_1000_results.Correct=='22']

In [ ]:
""" DONE

    Correct mark #23 #25:
        Закупівлі потрапляють до датафрейму так званих закупівель Управлінь освіти (відповідно до свого міста),
        оскільки визначити коректно назву неможливо
"""
# random_1000_results[random_1000_results.Correct=='23']

In [ ]:
""" DONE

    Correct mark #24:
        Переписати Step 7: додати пошук номеру в колонці ОписаниеТендера
"""
# random_1000_results[random_1000_results.Correct=='24']

In [7]:
""" UNDONE

    Correct marks #26: 
        Для шкіл-не-замовників у Step 8
        1. «» або ""
        2. відібрати перше? останнє? повторюване?

"""
# df_error_25_26 = random_1000_results[(random_1000_results.Correct=='25') | (random_1000_results.Correct=='26')]

,Тендер,ОписаниеТендера,Лот,Организатор,Тип закладу,Номер/Назва,Заклад-замовник,Correct
79710,послуги з поточного ремонту приміщення Черкась...,NaN,послуги з поточного ремонту приміщення Черкась...,Департамент освіти та гуманітарної політики Че...,Загальноосвітня школа,Невідомо,False,25
268691,Послуги з організації харчування (послуги з х...,КЗО СЗШ № 88,Послуги з організації харчування (послуги з х...,ДЕПАРТАМЕНТ ГУМАНІТАРНОЇ ПОЛІТИКИ ДНІПРОВСЬКОЇ...,Загальноосвітня школа,Невідомо,False,25
158767,ДК 021: 2015 код 45220000-5 «Інженерні та буді...,ДК 021: 2015 код 45220000-5 «Інженерні та буді...,ДК 021: 2015 код 45220000-5 «Інженерні та буді...,Управління освіти Дарницької районної в місті ...,Гімназія,Невідомо,False,25
49698,ДК 021: 2015 код 45450000-6 «Інші завершальні ...,відповідно до технічних вимог замовника,ДК 021: 2015 код 45450000-6 «Інші завершальні ...,Управління освіти Дарницької районної в місті ...,Гімназія,Невідомо,False,26
36721,Поточний ремонт по заміні віконних блоків у ЗЗ...,NaN,Поточний ремонт по заміні віконних блоків у ЗЗ...,Департамент освіти і науки Запорізької міської...,Навчально-виховний комплекс (об'єднання),Невідомо,False,25
112500,Електрична енергія,перша гімназія,Електрична енергія,Департамент економіки та розвитку Черкаської м...,Гімназія,Невідомо,False,25
250826,Послуги з проектно-вишукувальних робіт з обсте...,Послуги з проектно-вишукувальних робіт з обсте...,Послуги з проектно-вишукувальних робіт з обсте...,Управління освіти Шевченківської районної в мі...,Навчально-виховний комплекс (об'єднання),Невідомо,False,25
22166,Послуги щодо приєднання до електричних мереж ...,Послуги щодо приєднання до електричних мереж ...,Послуги щодо приєднання до електричних мереж ...,Відділ освіти Сихівського та Личаківського рай...,Навчально-виховний комплекс (об'єднання),Невідомо,False,25
14922,Поточний ремонт покрівлі за адресою: проспект ...,NaN,Поточний ремонт покрівлі за адресою: проспект ...,Управління освіти Голосіївської районної в міс...,Навчально-виховний комплекс (об'єднання),Невідомо,False,25
188937,Пензли та валики для господарських потреб НВК ...,NaN,Пензли та валики для господарських потреб НВК ...,Відділ освіти Сєвєродонецької міської ради,Навчально-виховний комплекс (об'єднання),Невідомо,False,25


In [ ]:
""" UNDONE

    Correct mark #3:
        Зміни у Step 3, Step 5:
            порядок filter_priority змінити
            
"""
# df_error_3 = random_1000_results[random_1000_results.Correct=='3']